# Selenium for 경제_무역뉴스

### Import Files

In [1]:
import sqlite3 as sq3
from pandas.io import sql
import os
import re
import string
import csv
import pandas as pd
from tabulate import tabulate
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import requests
import lxml.html
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

import pyperclip  # 클립보드 기능

## get_each_page()
- gets to each page

In [2]:
def get_each_page():
    article_details = []

    driver.get('https://dream.kotra.or.kr/dream/cms/com/index.do?MENU_ID=2370#;')
    click_the_first_link  = driver.find_element('xpath','//*[@id="tbody"]/tr[1]/td[2]/a').click()
    driver.implicitly_wait(8)

    while(True):
        title = driver.find_element('xpath','//*[@id="pdfArea"]/dl/dt/div[1]/h2').text
        print(title)
        article_details.append(get_info())
        try:
            p = driver.find_element('xpath','//*[@class="no_article"]').text
            # click previous button onless the article is the last one
            if (p == "이전글이 없습니다."):
                break
        except Exception:
            driver.implicitly_wait(1)
        click_prev = driver.find_element('xpath','//*[@id="contents"]/article/div/div[3]/ul/li[1]/p/a').click()
    return article_details

## remove_punc()
- removes punctuations from string

In [3]:
def remove_punc(data):

    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·「」△◈《》•◦ㅇ‘’中■○□※▷①②③④【】『∙』·-“”▲I]'
    new_string = re.sub(punc, '', data) # 특수문자 제거
    new_string2 = re.sub('\n', ' ', new_string) # newline 제거
    new_string3 = re.sub('\\s+', ' ', new_string2) # multiple spaces 제거
    return new_string3

## db_save()
- saves data as db file

In [4]:
def db_save(ARTICLE_LIST,db_name,table_name):
    with sq3.connect(os.path.join('.',db_name)) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            ARTICLE_LIST.to_sql(name = table_name, con = con, index = False, if_exists='replace') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(ARTICLE_LIST), '건 저장완료..')

In [5]:
def db_select(db_name, table_name):
    with sq3.connect(db_name) as con: 
        try:
            query = 'SELECT * FROM {}'.format(table_name)
            df = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e)) 
        return df

## get_info()
- returns detailed information on the article pages as list

In [6]:
def get_info():
    details = []
    article_s = []
    article_string = ""

    title = driver.find_element('xpath','//*[@id="pdfArea"]/dl/dt/div[1]/h2').text
    category = driver.find_element('xpath','//*[@id="pdfArea"]/dl/dt/div[2]/ul/li[1]/span').text
    country = driver.find_element('xpath','//*[@id="pdfArea"]/dl/dt/div[2]/ul/li[2]').text
    articles = driver.find_elements('xpath','//*[@class="viewDataWrap"]/p')
    for article in articles:
        article_s.append(article.text)
    article_string = " ".join(article_s)
    article_edited = remove_punc(article_string)
    
    details.append(title)
    details.append(category)
    details.append(country)
    details.append(article_edited)
    #details.append(get_pdf())
    
    return details

In [7]:
def get_pdf():
    # click download and wait
    articles = driver.find_element('xpath','//*[@id="pdfArea"]/dl/dd/div[1]/ul/li/a').click()
    article_title = driver.find_element('xpath','//*[@id="pdfArea"]/dl/dd/div[1]/ul/li/a').text
    driver.implicitly_wait(15)
    
    article_title2 = re.sub(' ','%20', article_title)
    article_title3 = 'file:///C:/Users/user/Downloads/'+ article_title2
    driver.get(article_title3)
    
    # action chain object creation
    a = ActionChains(driver)
    # perform the ctrl+c pressing action
    select_a = a.key_down(Keys.CONTROL).send_keys('A').perform()
    copy_a = a.key_down(Keys.CONTROL).send_keys('C').perform()
    articles = pyperclip.paste()
    print(articles)
    
    for eachstring in articles:
        article = ' '.join(eachstring.text)
    article = re.sub('  ', ' ', article)
    
    driver.implicitly_wait(4)
    
    words = remove_punc(article)
    driver.close()
    return words

## db_save_as_csv()
- saves dataframe as csv

In [8]:
def db_save_as_csv(data):
    data.to_csv("경제_무역.csv", index=False, encoding='utf-8')

### ---------------------------------------------------------------------------------------------------------------------
# Main
This is the main part which uses functions to read articles and saves the data as csv file
### ---------------------------------------------------------------------------------------------------------------------

In [9]:
options = webdriver.ChromeOptions() 
#options.add_argument("--start-maximized");  # Chrome 브라우저 최대화 설정
options.add_argument('--headless')  # headless 모드
options.add_argument('--disable-gpu')  # 그래픽 가속 해제 (오류 방지)
#options.add_argument('--mute-audio')  # 음소거모드 적용

driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = options)

driver.get('https://dream.kotra.or.kr/dream/cms/com/index.do?MENU_ID=2370')
driver.implicitly_wait(5)

df_list = []
details = get_each_page()

for each_det in details:
    df = pd.DataFrame([{
        "제목": each_det[0],
        "카테고리": each_det[1],
        "국가": each_det[2],
        "본문": each_det[3]
    }])
    df_list.append(df)
    driver.implicitly_wait(4)
    
ARTICLE_LIST = pd.concat(df_list)

db_save(ARTICLE_LIST,'경제_무역.db','경제_무역')
db_save_as_csv(ARTICLE_LIST)

driver.close()
driver.quit()

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.68M/6.68M [00:00<00:00, 11.9MB/s]


[EU 경제통상 리포트(22-26)] EU, 에너지 위기 대응 긴급 조치 발표
[미국경제통상리포트(US22-33)]美 연방준비은행 페드나우(Fed Now) 출시 전망 및 시사점
숫자로 보는 세계 70개 항구 적체 현황 리포트(9월 호)
미국 국가 생명공학 및 바이오 제조 이니셔티브(9.12) 주요 내용
[미국경제통상리포트(US22-32)]미국의 전기차 배터리용 핵심 광물 확보 전략
[미국경제통상리포트(US22-31)] IPEF 9월 장관회의 세부 내용 및 현지 반응
인도 정부의 탄소배출 감축 계획과 우리기업 기회요인
KOTRA IP-DESK 지식재산권 뉴스레터 7호(ASEAN+)
2022년 8월 글로벌 및 주요국 경제동향과 전망
[2022.09.08] 글로벌 공급망 인사이트 27호
케냐 대선(8.9) 결과 및 향후 전망
KOTRA IP-DESK 지식재산권 뉴스레터 6호(IP5+)
영국 총리 경선 결과 및 전망
[미국경제통상리포트(US22-30)] 반도체 칩과 과학법 세부 내용 및 시사점
[미국경제통상리포트(US22-29)] 미국 2분기 GDP 주요 지표 동향 및 시사점
[미국경제통상리포트(US22-28)] 미국 6월 주택시장 동향 및 전망
[미국경제통상리포트(US22-27)] 미리 보는 美 의회 중간선거 결과와 관전포인트
[EU 경제통상 리포트(22-25)] EU 디지털 패키지(DMA&DSA) 입법 동향과 시사점
[EU 경제통상리포트(22-24)] 유럽 물류시장 동향 및 업계 대안
[EU 경제통상리포트(22-23)] 러시아-우크라 사태 관련 EU 정책 동향 및 전망
2022년 상반기 우리나라의 전자상거래 수출 동향
[2022.09.01] 글로벌 공급망 인사이트 26호
[2022.08.25] 글로벌 공급망 인사이트 25호
KOTRA IP-DESK 지식재산권 뉴스레터 4호(IP5+), 5호(ASEAN+)
[미국경제통상리포트(US22-26)] 미국 인플레이션 감축법 입법동향 및 주요내용
숫자로 보는 세계 70개 항구 적체 현황 리포트(8월 호)
[2022.08.18

신남방지역 주요 이슈 리포트(2021년 8월 12일)
신남방지역 주요 이슈 리포트(2021년 8월 5일)
신남방지역 주요 이슈 리포트('21.9.29~8.4)
중국 상반기 경제지표 평가와 전망
중국 6월 및 상반기 수출입 동향 및 전망
EU 탄소국경조정제도(CBAM) 규정안 주요내용 요약
중국 1~5월 신에너지 승용차 판매 급증과 시사점
中 공산당 창당 100주년 시진핑 연설 주요 내용
美 공급망 검토 주요내용 및 향후전망
美 정부 자국산 우대 조달 정책 추진동향
美 상원 통과 미국혁신경쟁법안 주요내용 요약
G20 경제현황 및 전망
중국 제7차 인구센서스 결과 발표
일본의 탄소 프라이싱 정책 추진과 전망
디지털위안화 시범운영 현황 및 주목할 유망산업
바이든 대통령 상하원 합동연설 요약
중국 통상정책 방향 전망 주요내용
글로벌 전기차 배터리 시장 동향 및 주요 이슈
사우디 지역본부유치(RHQ) 정책 주요내용 및 시사점
유럽 주요 자동차 제조사 전기차 전환 동향 및 전망
기후 정상회의 개최 결과 요약
중국 1분기 경제지표 평가
美 재무부 환율정책 보고서 발표 주요내용 요약
설문조사로 살펴본 2021년 아세안 주요 이슈 및 시사점
美 바이든 정부의 글로벌 최저법인세율 제안 및 전망
일본기업의 GVC 재구축 추진현황 및 대응사례
美정부 인프라 제조업 등 투자를 통한 경제재건 계획
미국 백신 공급 계획에 따른 경제 전망
EU탄소국경조정제도 논의 동향과 추진전망
바이든 행정부 미국 최초로 환율 상계관세 부과 결정
코로나19 악화일로 브라질의 재확산 현황 및 전망
글로벌 희토류 산업동향 및 시사점
바이든 정부 기후변화대응 정책동향 및 전망
중국 1~2월 경제지표로 본 경기회복세 평가
中 양회를 통해 본 정책방향과 주목할 3대 산업
미국 EU정부의 미래산업 공급망 추진동향 보고
최근 중국의 세계교역 점유율 증가와 韓中 교역 관계 변화
2021 미국 통상정책 아젠다 요약보고
유럽 그린딜 주요 내용 및 시사점
일본 수소사회 구현을 위한 국제 수소GVC 구축
2021 중국 전국인

## reads the saved csv file and displays the result as a dataframe

In [10]:
document = db_select('경제_무역.db','경제_무역')

In [11]:
document

제목         카테고리        국가  \
0          [EU 경제통상 리포트(22-26)] EU, 에너지 위기 대응 긴급 조치 발표  글로벌 이슈 모니터링       벨기에   
1    [미국경제통상리포트(US22-33)]美 연방준비은행 페드나우(Fed Now) 출시 ...  글로벌 이슈 모니터링        미국   
2                     숫자로 보는 세계 70개 항구 적체 현황 리포트(9월 호)  글로벌 이슈 모니터링  KOTRA 본사   
3                미국 국가 생명공학 및 바이오 제조 이니셔티브(9.12) 주요 내용  글로벌 이슈 모니터링        미국   
4         [미국경제통상리포트(US22-32)]미국의 전기차 배터리용 핵심 광물 확보 전략  글로벌 이슈 모니터링        미국   
..                                                 ...          ...       ...   
292                                    2020년 중국 주요경제지표  글로벌 이슈 모니터링        중국   
293                              2020년 중국 대외교역 현황 및 전망  글로벌 이슈 모니터링        중국   
294                               미국 디지털 화폐 추진 동향 및 전망  글로벌 이슈 모니터링        미국   
295                   중국 외국인투자 장려산업 목록 주요 내용과 평가 및 시사점  글로벌 이슈 모니터링        중국   
296                   조지아 상원의원 선거 결과와 바이든 정부의 경제 정책 전망  글로벌 이슈 모니터링        미국   

                                                    본문  
0     EU 집행위 러시아發 에너지 위기 대응 위해 긴급 조치 발표914 수요 감축 의무...  
1     모든 금융기관 대상 9월 시범 도입 및 내년 57월 전격 출시 전망 페드나우 도입...  
2    KOTRA는 A빅데이터를 기반으로 해상운송 정보를 제공하는 스타트업 씨벤티지SeaV...  
3     바이든 대통령은 바이오 산업의 자국내 생산 강화를 골자로 하는국가 생명공학 및 바...  
4     인플레이션 감축법으로 전기차EV 산업 재편 가속화 미국 내 EV 배터리 생산 확대...  
..                                                 ...  
292  2020년 중국 주요경제지표 2020년 중국 경제성장률 23 GDP 총액 100조 ...  
293  2020년 중국 대외교역 현황 및 전망 수출입 현황 20년 교역액 전년 대비 15 ...  
294  미국 디지털 화폐 추진 동향 및 전망 최근 동향 연방정부 중국의 디지털 위안 도입 ...  
295  중국 외국인투자 장려산업 목록 주요 내용과 평가 및 시사점 외국인투자 장려산업 목록...  
296  조지아 상원의원 선거 결과와 바이든 정부의 경제 정책 전망 선거결과 높은 우편 투표...  

[297 rows x 4 columns]